In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np

from datetime import date
from datetime import timedelta
from sklearn import preprocessing

module_path = os.path.abspath("/seq/vgb/bryc/projects/pheno")
sys.path.insert(0, module_path)

import pheno
from pheno import PhenoDB

In [ ]:
age_map = {
    '2018-12-08': np.nan,
    '2016-10-30': np.nan,
    '2017-03-03': np.nan,
    '8-10': "9",
    'born about 2011' : np.nan,
    '9 years': "9",
    '2017-02-16' : np.nan,
    '3 1/2': "3.5",
    '2017-06-24' : np.nan,
    '1 year': "1",
    '12-13 years old': "12.5", 
    '2ish': "2", 
    '2 1/2': "2.5",
    '1-2 yrs': "1.5",
    '8 years': "8",
    '4-5 years': "4.5",
    '2017-11-26' : np.nan,
    '2009-09-01' : np.nan, 
    '2016-01-01' : np.nan,
    '2011-05-29' : np.nan,
    '2012-01-05' : np.nan,
    '2013-03-01' : np.nan,
    '3.5 yrs': "3.5",
    '5 months': "0.42",
    'about 9': "9", 
    '5 ?': "5",
    '12 weeks':"0.25" ,
    '5-6': "5.5", 
    '3 years': "3", 
    '4;&#39;': "4",
    '10months': "0.83", 
    '7-10': "8",
    '011 months': "0.92",
    '23 months': "1.92",
    '123 months': np.nan,
    '1 yr, 4 mos':"1.33", 
    '5m': "0.42", 
    '9 months' : "0.75",
    '6th': np.nan, 
    '6 months': "0.5",
    '1year':"1.0",
       '1.5 yr': "1.5",
    '11?': "11",
    '10 mos': "0.83",
    '11 months': "0.92",
    'about 15': "15", 
    '19 weeks': "0.4" ,
    '20 months': "1.67",
    '3ish': "3", 
    '08': np.nan,
    '13ish': "13",
    '3.5 months':"0.29",
    '5 mo': "0.42",
    '3 yr': "3",
    '~10 mo.':"0.83",
    '15?': "15",
    '3 +/-': "3",
    '10 months': "0.83",
    'About 7 years, 10 months': "7.83",
    '9+ years': "9",
    '~5': "5",
    '12 months': "1.0",
    'approx 7 years': "7",
    '9 mos': "0.75",
    '16mo': "1.33",
    '06 months': "0.5",
    '7mos.': "0.58",
    '9mo': "0.75",
    '2-ish': "2", '55':np.nan,
    '16 months': "1.33",
    '20 mos': "1.67", 
    '?':np.nan,
    '07 months':"0.58",
    '66':np.nan, 
    '10-12': "11",
    '6 mo': "0.5",
    '6 + / -':"6",
    'NOT SURE 300 YEARS REINCARNATED':np.nan,
    '6+9mos': "6.82",
    '77': np.nan,
    '15ish': "15",
    '8?': "8",
    '8 months': "0.67",
    '03': "0.3",
    '33': np.nan,
    '22': np.nan, 
    '1y7mos': "1.58",
    '9-10 yo': "9.5",
    '10?': "10",
    '7 weeks': "0.15",
    '1 1/2': "1.5",
    '5 1/2': "5.5",
    '4y/5m': "4.42",
    '8mo': "0.67",
    '9 weeks': "0.19",
    '01': np.nan,
    '9mos': "0.75",
    '6mo': "0.5",
    '07': "7",
    '6 YO': "6",
    '5 mos': "0.42",
    '2014': np.nan,
    '7 1/2 yrs old': "7.5",
    "5 or 6 years": "5.5",
    "2 1/2 years": "2.5",
    "5 to 7 years old": "6.0",
    "6 or 7 years": "6.5"
}

In [ ]:
cache_dir = "/seq/vgb/bryc/data/darwins_dogs/2024-09-23_cache"
ph = PhenoDB(cache_dir)

In [ ]:
# Find the SRA uploaded dog IDs and subset to those
SRA_file = "/seq/vgb/rawData/gencove/DarwinsArk/fastq/SRA/sampleIDs.txt"
filenames = pd.read_csv(SRA_file, header=None)
filenames[['DA', 'dogID', 'barcode']] = filenames[0].str.split('_', n=2, expand=True)
filenames['dogID'] = filenames.dogID.astype(int)
# Fix two incorrect SRA file dogIDs (filenames were incorrect, dog is correct in spreadsheet/data):
filenames["dogID"] = filenames["dogID"].replace({26971: 26961, 12450:12350})
SRA_2024 = pd.read_csv("/seq/vgb/rawData/gencove/DarwinsArk/fastq/2024-08-22_SRA/files/2024-08-22_SRA.csv")
all_SRA_dogs = (SRA_2024['darwinsark_id'].tolist() + filenames['dogID'].tolist())


In [ ]:
print(len(all_SRA_dogs))

In [ ]:
# Cleanup of sex, neuter, purebred, and age
subset = ph.dogs[ph.dogs.id.isin(all_SRA_dogs)].copy()
subset["sex"] = subset['sex'].replace({"Male": "male", "Female": "female"})
subset["neutered"] = subset['neutered'].replace({"Yes": "yes", "No": "no"})
subset["purebred"] = subset["purebred"].replace({"1": "yes", "0": "no", "-1": np.NaN})
subset["age"] = subset["age"].replace(age_map)
# Remove non-plausible ages
subset.loc[subset.age_norm > 25, "age_norm"] = np.nan

### Get `age_composed` which will combine the best information across columns

In [ ]:
subset['creation_date']= pd.to_datetime(subset['creation_date'])
subset['birthday']= pd.to_datetime(subset['birthday'])
subset["deceased_date"] = pd.to_datetime(subset['deceased'], unit='s')

In [ ]:
subset["inferred_age"] = (subset['creation_date'] - subset['birthday']) / timedelta(days=365)
subset["numeric_age"] = pd.to_numeric(subset["age"], errors='coerce')
subset["age_at_death"] = (subset['deceased_date'] - subset['birthday']) / timedelta(days=365)
subset["age_composed"] = subset["age_norm"].fillna(subset["inferred_age"])
subset["age_composed"] = subset["age_composed"].fillna(subset["numeric_age"])

In [ ]:
age_cols = ["id", "age", "creation_date", "birthday", "age_composed", "age_norm", "birthday_historical", "inferred_age", "numeric_age", "age_at_death"]
subset[subset.age_norm - subset.inferred_age > 0.5][age_cols]

# Overwrite with the manually entered age because there is a typo in the birthyear
subset.loc[subset.id == 2144, "age_composed"] = 4.0
subset.loc[subset.id == 11503, "age_composed"] = 3.0

### Get breed information

In [ ]:
id_to_breed = dict(zip(ph.breeds.id, ph.breeds.breed_print))
for column in ['breed1_id','breed2_id', 'breed3_id']:
    subset[column] = subset[column].replace(id_to_breed)
subset[['breed1_id','breed2_id', 'breed3_id']]

In [ ]:
# Get combined breed information between older freetext data and recent ID columns
subset['breed1'] = subset['breed1'].fillna(subset['breed1_id'])
subset['breed2'] = subset['breed2'].fillna(subset['breed2_id'])
subset['breed3'] = subset['breed3'].fillna(subset['breed3_id'])

# Clean up messy manually-entered breed names
manual_fixes = {"beagle \"plus\" ... tri-color white + liver + brown; brown \"freckles\" on white legs": "Beagle",
               "sato": "Sato",
               "miniature australian shepherd": "Miniature Australian Shepherd",
               "Black mouth curr": "Black Mouth Cur",
               "wire haired fox terrier": "Wire Fox Terrier",
               "Wire Haired Fox Terrier": "Wire Fox Terrier",
               "Congo dog": "Basenji",
               "Swiss White Shepherd":"White Swiss Shepherd Dog",
               "Hamiltonstovare": "Hamilton Hound",
               "labradoodle": "Labradoodle",
               "50% Chihuahua ":"Chihuahua",
               "50% Australian Kelpie ": "Australian Kelpie",
               "Akbash Dog": "Akbash",
               "Berger Blanc Suisse": "White Swiss Shepherd Dog",
               'German Munsterhund': "Small Münsterländer",
               "Xoloitzcuintli": "Mexican hairless dog",
               "Black & Tan Coonhound": "Black and Tan Coonhound",
               "Louisiana Catahoula Leopard Dog": "Catahoula Leopard Dog",
               "German Shepherd Dog (50%)": 'German Shepherd Dog',
               "windsprite": "Silken Windsprite",
               "Golden Retriever?": "Golden Retriever",
               "miniature dachshund": "Miniature Dachshund",
               "American Black and Tan Coonhound": "Black and Tan Coonhound",
               "Plott": "Plott Hound",
               "Red Heeler": "Australian Cattle Dog",
               "Flat-coated Retrievers": "Flat-Coated Retrievers",
               "Blue Heeler": "Australian Cattle Dog",
               "mixed":np.nan,
                "Mix": np.nan,
               "tennesee brindle mountain cur": "Treeing Tennessee Brindle",
               "Chihuahua (Long": "Chihuahua (Long Coat)",
               " Boarder Collie ": "Border Collie",
               "25% GSD": "German Shepherd",
               "25% Chow": "Chow Chow",
               "larger hound": np.nan,
                "australian kelpie": "Australian Kelpie",
               "catahoula leopard dog": "Catahoula Leopard Dog"}
subset["breed1"]= subset.breed1.replace(manual_fixes)
subset["breed2"]= subset.breed2.replace(manual_fixes)

In [ ]:
# Use breed2 results if breed1 is missing
subset['breed1'] = subset['breed1'].fillna(subset['breed2'])
subset.loc[subset.breed1 == subset.breed2,'breed2'] = np.nan

### Get ancestry results for all dogs

In [ ]:
ancestry = pd.read_csv(os.path.join(cache_dir, "dog_breed.csv"))
ancestry = ancestry[ancestry.version == "v2"]
ancestry["breed"] = ancestry["breed"].replace(id_to_breed)
samples  = pd.read_csv(os.path.join(cache_dir, "samples.csv"))
non_na_samples = samples[~samples.dog.isna()]
barcode_to_dog = dict(zip(non_na_samples.barcode, non_na_samples.dog))
dog_to_barcode = {v: k for k, v in barcode_to_dog.items()}
ancestry["id"] = ancestry["barcode"].map(barcode_to_dog)
with_ancestry = ancestry.id.unique()

In [ ]:
missing = [dog  for dog in subset.id if dog not in with_ancestry]
print(len(missing))

In [ ]:
barcode_fixes = {}
barcodes_with_anc_results = ancestry.barcode.unique()

for dog in missing:
    try:
        barcode = dog_to_barcode[dog]
        if (barcode + "-a") in barcodes_with_anc_results:
            barcode_fixes[barcode + "-a"] = dog
        else:
            print("No results for barcode", barcode, "and dog", dog)
    except:
        print("dog ", dog, " not found in samples table")

ancestry["id"] = ancestry["id"].fillna(ancestry["barcode"].map(barcode_fixes))

In [ ]:
# Side bar: let's make a file to address these 128 mismapped dogs and give them breed results
import time

unix_timestamp = int(time.time()) 
mismapped_barcode_file = "/seq/vgb/rawData/gencove/raw/doc/2024-09-30_Kasia_mismapped_barcode_fixes.txt"
with open(mismapped_barcode_file, "w") as file1:
    for barcode, dog in barcode_fixes.items():
        # Writing data to a file
        file1.write(f"insert into samples (time,type,barcode,dog,location1,project) values ({unix_timestamp},1,\"{barcode}\",{dog},17,1);\n")


In [ ]:
with_ancestry = ancestry.id.unique()
still_missing = [dog  for dog in subset.id if dog not in with_ancestry]

print(len(still_missing))

In [ ]:
to_fix = {}
for dog in filenames[filenames.dogID.isin(still_missing)].dogID.values:
    try:
        bar = dog_to_barcode[dog]
        to_fix[dog] = bar
    except:
        print("no barcode for dog ", dog)

In [ ]:
dedup = ancestry.drop_duplicates(keep='last', subset = ["id", "breed"])
df = dedup.pivot(index='id', columns='breed', values='percent').reset_index()
ancestries = df[df.id.isin(subset.id)]

### Prepare merged dataframe

In [ ]:
useful_cols = ['id',
 'name',
 'sex',
 'neutered',
 'creation_date',
 'age_composed',
 'age',
 'age_norm',
 'numeric_age',
 'inferred_age',
 'dob_known',
 'birthday',
 'birthday_historical',
 'breed1',
 'breed2',
 'breed3',
 'purebred',
 'deceased_date',
 'age_at_death']

In [ ]:
subset[useful_cols] 

In [ ]:
# merge phenotypes with ancestries
ancestries = ancestries.fillna(0)
phenotypes = pd.merge(subset[useful_cols], ancestries, how = "outer", on="id")

In [ ]:
phenotypes

In [ ]:
breed = "Boxer"
phenotypes[phenotypes.breed1 == breed][breed].hist(bins = 100)

## Compute behavior factors

In [ ]:
factor_loadings=pd.read_csv("/seq/vgb/bryc/projects/dog_behavioral_gwas_paper/data/DarwinsArk_20221120_factor-analysis_discovery-no-na_qn-182_dn-7883_fn-25.loadings.tsv", sep="\t")

In [ ]:
answers = ph.df.reset_index().rename(columns={"dog": "id"})
answers = answers[answers.id.isin(all_SRA_dogs)]

In [ ]:
# Standardize the input data for a range of 0 to 6
all_qs= factor_loadings.question.unique()
columns = ["id"] + list(all_qs)
scaler = preprocessing.MinMaxScaler(feature_range=(0, 6))
scaled = scaler.fit_transform(answers[all_qs])
scaled_features_df = pd.DataFrame(scaled, index=answers.index, columns=all_qs)

In [ ]:
for factor in factor_loadings.factor.unique():
    loadings = factor_loadings[factor_loadings.factor == factor][['question', 'pattern']].set_index('question')
    df = scaled_features_df[loadings.index]
    answers["factor_" + str(factor)] = df.dot(loadings.pattern)

### Merge dataframes

In [ ]:
answers.factor_16.hist(bins=100)

In [ ]:
all_phenotypes = pd.merge(phenotypes, answers, on="id", how = "left")

In [ ]:
all_phenotypes.to_csv("/seq/vgb/bryc/data/darwins_dogs/2024-09-23_cache/merged_phenotype_table.csv", index=False)